# TODO
 - [x] Compare Death Counts by State
 - [x] Compare Death Counts across all states and show TCTP and CDC data for deaths, differences (CDC - TCTP) and % difference then sort by % difference
 - [x] Merge two cdc and CovidTracking data into one DataFrame
 - [x] add difference column for appropriate fields
 - [x] add percent difference column for appropriate fields
 - [ ] Read data of data from CDC for proper comparison

In [1]:
import pandas as pd

In [2]:
# Get Covid Tracking Project Data indexed by state abbreviation
#tctpDF = pd.read_csv('https://covidtracking.com/api/v1/states/current.csv', index_col='state')
tctpDF = pd.read_csv('https://covidtracking.com/api/v1/states/current.csv')

# Get CDC Data indexed by state abbreviation
#cdcDF = pd.read_csv('https://www.cdc.gov/covid-data-tracker/Content/CoronaViewJson_01/US_MAP_DATA.csv', skiprows=2, index_col='abbr')
cdcDF = pd.read_csv('https://www.cdc.gov/covid-data-tracker/Content/CoronaViewJson_01/US_MAP_DATA.csv', skiprows=2)

In [3]:
# tctpDF.join(cdcDF.set_index('state'), on='state')
#pd.concat([cdcDF, tctpDF], axis=1).head(3)
newDF = pd.merge(cdcDF.add_prefix('CDC_').rename(columns={'CDC_abbr': 'state'}), tctpDF, on='state', how='outer', sort=True, copy=True)

In [4]:
columns = ['state', 'CDC_fips', 'CDC_jurisdiction', 'CDC_Total Cases', 'CDC_Total Death', 'CDC_CasesInLast7Days', 'CDC_RatePer100000', 'CDC_Total Tests Performed', 'CDC_Percent Positive Range', 'CDC_cumulative_positive_tests',
           'positive', 'negative', 'pending', 'death', 'total', 'totalTestResults', 'posNeg', 'fips','hash']
diffDF = newDF[columns].copy()

In [7]:
# Map CDC fields to TCTP fields - key is CDC field name and value is Covid Tracking Project field name
#     It is a list of tuples with CDC Field Name, TCTP Field Name and Label for new column
fieldMappings = [('CDC_Total Death', 'death', 'Deaths'),
                 ('CDC_Total Cases', 'positive', 'Cases-Positives')]
for item in fieldMappings:
    cdcField = item[0]
    tctpField = item[1]
    label = item[2]
    diffDF['Diff_' + label] = diffDF[cdcField] - diffDF[tctpField]
    diffDF['Pct_Diff_' + label] = (diffDF['Diff_' + label] / diffDF[cdcField] ) * 100

#         pctDiff = (diff / cdcVal) * 100
diffDF[['CDC_Total Death', 'death', 'Diff_Deaths', 'Pct_Diff_Deaths']].head(3)
diffDF.to_csv('covid_CSC_vs_CovidTracking.csv', index=False)

In [6]:
diffDF

,state,CDC_fips,CDC_jurisdiction,CDC_Total Cases,CDC_Total Death,CDC_CasesInLast7Days,CDC_RatePer100000,CDC_Total Tests Performed,CDC_Percent Positive Range,CDC_cumulative_positive_tests,...,death,total,totalTestResults,posNeg,fips,hash,Diff_Deaths,Pct_Diff_Deaths,Diff_Cases-Positives,Pct_Diff_Cases-Positives
0,AK,2,Alaska,383,10,13,51.9,44788.0,0-5%,NaN,...,10.0,30649.0,30649.0,30649.0,2.0,65fc900d3f18980bc2b5ce61bc063910f001f945,0.0,0.000000,0.0,0.000000
1,AL,1,Alabama,10665,433,2353,218.2,326258.0,11-20%,NaN,...,449.0,136372.0,136372.0,136372.0,1.0,fa40f8ecf2c6c2d24de9fe2e32e171ce45f9c457,-16.0,-3.695150,48.0,0.450070
2,AR,5,Arkansas,4164,95,695,138.2,78291.0,6-10%,NaN,...,95.0,73215.0,73215.0,73215.0,5.0,f147419663f32a125b49bd869ff47e8e6742cb30,0.0,0.000000,0.0,0.000000
3,AS,60,American Samoa,0,0,0,0.0,NaN,NaN,NaN,...,0.0,105.0,105.0,105.0,60.0,d55efc5d7e5e12f7f4e5262ee6282043f7d2f2e7,0.0,NaN,0.0,NaN
4,AZ,4,Arizona,11736,562,2817,163.6,192589.0,6-10%,NaN,...,594.0,127750.0,127750.0,127750.0,4.0,be079f12d747ed3e1f603133ebdd1131748b5572,-32.0,-5.693950,-440.0,-3.749148
5,CA,6,California,69382,2847,14445,175.4,861539.0,6-10%,NaN,...,2934.0,1065592.0,1065592.0,1065592.0,6.0,d02741729ab28ef93af87a37e2308fd075292a66,-87.0,-3.055848,-1759.0,-2.535240
6,CO,8,Colorado,20091,1006,3213,352.7,158078.0,11-20%,NaN,...,1009.0,111250.0,111250.0,111250.0,8.0,634ca84bc70d7edd003c2afc35e1d5e6956cf04d,-3.0,-0.298211,-66.0,-0.328505
7,CT,9,Connecticut,34333,3041,4360,961.0,140434.0,21-30%,NaN,...,3125.0,142943.0,142943.0,142943.0,9.0,cb2f89a519afa3ca40138fed30153091eafaab7e,-84.0,-2.762249,-522.0,-1.520403
8,DC,11,District of Columbia,6485,336,1315,923.2,30821.0,21-30%,NaN,...,350.0,31658.0,31658.0,31658.0,11.0,0eb63fe6dca1a46d64cd8ede20cb5c3c38483df7,-14.0,-4.166667,-99.0,-1.526600
9,DE,10,Delaware,6952,247,1664,718.8,31227.0,11-20%,NaN,...,247.0,34753.0,34753.0,34753.0,10.0,8f9d800ca5c57df79f7ab84b54bfe019f51a77e8,0.0,0.000000,0.0,0.000000


# Play Area